# 机器学习因果推断——Stage2——模型2

多项机器学习算法对比，利用shap方法获得解释性

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np


## Read Data

In [ ]:
pred_path = r"C:\Software\Local Things (Coding)\comp\2025大学生建模比赛\代码\比赛项目代码\CNN_prediction.csv"
stock_vol_path = r"C:\Software\Local Things (Coding)\comp\2025大学生建模比赛\数据\stock_volatility.csv"
index_list = ['DJI', 'IXIC', 'NDX', 'NYA', 'NYID', 'OEX', 'SPX']
pred_data = pd.read_csv(pred_path)
stock_vol_data = pd.read_csv(stock_vol_path)
stock_vol_data.rename(columns={"交易日期": "time"}, inplace=True)

print(f"prediction data shape: {pred_data.shape} \n prediction data columns: {pred_data.columns}")
print(f"stock volatility data shape: {stock_vol_data.shape} \n stock volatility data columns: {stock_vol_data.columns}")
print(stock_vol_data[:3])


In [ ]:
stock_vol_data.hist(bins=100, figsize=(10, 6))

## 先对于预测数据先把预测值取均值

In [ ]:
pred_df = pred_data[['time', 'name', 'prob_class_0', 'prob_class_1', 'prob_class_2', 'prob_class_0.1', 'prob_class_1.1', 'prob_class_2.1']]
# netural == 1 negative == 0 positive == 2
pred_df['prob_negative'] = 1/2 * pred_df['prob_class_0'] + 1/2 * pred_df['prob_class_0.1']
pred_df['prob_neutral'] = 1/2 * pred_df['prob_class_1'] + 1/2 * pred_df['prob_class_1.1']
pred_df['prob_positive'] = 1/2 * pred_df['prob_class_2'] + 1/2 * pred_df['prob_class_2.1']
pred_df_select = pred_df[['time','name', 'prob_negative', 'prob_neutral', 'prob_positive']]
pred_df_select.to_csv('integrated_pred.csv',index=False,encoding='utf-8-sig')
pred_df_select.head()



In [ ]:
# wide => long table for each probs

df = pred_df_select.copy()
df['time'] = pd.to_datetime(df['time'],errors='coerce')
df['time'] = df['time'].dt.date
sentiment_label = 'prob_negative'
# sentiment_label = 'prob_neutral'
# sentiment_label = 'prob_positive'

# [0.21195608 0.57607543 0.21196853]
# df_final = df[["time", "name", "prob_positive"]]
if sentiment_label == 'prob_negative':
    imputation = 0.21195600
elif sentiment_label == 'prob_neutral':
    imputation = 0.57607543
elif sentiment_label == 'prob_positive':
    imputation = 0.21196853

df_final = df[["time", "name", sentiment_label]]

df_grouped = df_final.groupby(['time', 'name'], as_index=False).agg({sentiment_label: 'mean'})

negative = df_grouped.pivot_table(index='time', columns='name', values=sentiment_label, aggfunc='mean')
negative = negative.reset_index()

negative.head()

In [ ]:
print(stock_vol_data['time'][:3])
stock_vol_data['time'] = pd.to_datetime(stock_vol_data['time']).dt.date
print(stock_vol_data['time'][:3])

In [ ]:

full_data = pd.merge(negative,stock_vol_data, on = 'time',how='outer')
full_data.drop(index = [0,1],inplace=True)
full_data.dropna(subset=index_list,inplace=True)
full_data.reset_index(drop=True,inplace=True)
full_data.to_csv('regression_data.csv',index=False,encoding='utf-8-sig')
full_data.head()

## Preprocess Data

In [ ]:
import pandas as pd
data = pd.read_csv('regression_data.csv')
data.set_index('time', inplace=True)
print(data.columns)
data.head()

## EDA

In [ ]:
from matplotlib import pyplot as plt


cols = [ 'IXIC', 'NDX', 'DJI','BillGates', 'Eyalo365', 'JackMa', 'JeffBezos', 'MichaelDell',
       'MukeshDhiAmbani', 'RayDalio', 'RicardoBSalinas', 'ShivNadarFDN',
       'StevenACohen2', 'Steven_Ballmer', 'elonmusk', 'ericschmidt',
       'esaverin', 'laurenepowell', 'leijun', 'lemannoficial',
       'mackenziescott', 'masason', 'mcannonbrookes', 'scottfarkas',
       'udaykotak',]
data[cols].hist(bins=50, figsize=(15,15))
plt.savefig('hist.png')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
corr = data[cols].corr()
sns.heatmap(corr, annot=False, cmap='coolwarm',linewidths=0,annot_kws={"size": 10})
plt.savefig('corr.png')
plt.show()


In [ ]:
data[cols].describe().T.to_csv('data_describe.csv',float_format='%.3f')

# ML model for stage 2

We plan to use the following models:

- Lasso
- Ridge
- Elastic Net
- Random Forest
- Bagging
- XGBoost

思路首先是对于单一指数进行回归，IXIC为基准

然后对于其他指数回归，作为稳健型检验

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge,Lasso,ElasticNet,LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error, r2_score
import multiprocessing

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import xgboost as xgb

# [0.21195608 0.57607543 0.21196853]

data.fillna(imputation,inplace=True)
threads = multiprocessing.cpu_count()-2
X = data.drop(index_list, axis=1)
# y = data['IXIC']
y = data['NDX']

print(X.shape, y.shape)
print(X.columns)
print(y.head())


In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, median_absolute_error, r2_score,mean_absolute_percentage_error
import matplotlib.pyplot as plt
import pandas as pd
param_grids={
    'Ridge':{
        'ridge__alpha':[ 0.01, 0.1, 1, 10, 100,]
    },
    'Lasso':{
        'lasso__alpha':[ 0.01, 0.1, 1, 10, 100]
    },
    'ElasticNet':{
        'elasticnet__alpha':[ 0.01, 0.1, 1, 10, 100],
        'elasticnet__l1_ratio':[0.1,0.3,0.5,0.7,0.9]
    },
    'SVR':{
        'svr__C': [0.01,0.1, 1, 10, 100],
        'svr__kernel': ['rbf'],
        'svr__gamma': [0.01,0.1, 1, 10, 100]
    },
    'Decision_Tree':{
        'decision_tree__max_depth': [2, 4, 6, 8, 10, 12, 14],
        'decision_tree__min_samples_split': [2, 5, 10, 20]
    },
    'Random_Forest':{
        'random_forest__max_depth': [2, 4, 6, 8, 10, 12, 14],
        'random_forest__min_samples_split': [2, 5, 10, 20],
        'random_forest__n_estimators': [10, 50, 100, 200]
    },
    'xgb':{
        'xgb__max_depth': [2, 4, 6, 8, 10, 12, 14],
        'xgb__min_child_weight': [1, 2, 3, 4, 5],
        'xgb__learning_rate': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9],
        'xgb__n_estimators': [10, 50, 100, 200]
    }
}

models={
    'OLS': LinearRegression(),
    'Ridge': Ridge(random_state=666),
    'Lasso': Lasso(random_state=666),
    'ElasticNet': ElasticNet(random_state=666),
    'SVR':SVR(),
    'Decision_Tree': DecisionTreeRegressor(),
    'Random_Forest': RandomForestRegressor(),
    'xgb': xgb.XGBRegressor()
    
}


In [ ]:

results = pd.DataFrame(columns=['Model', 'Best Params', 'RMSE', 'MAE', 'MeAE', 'R-squared'])
coef_df = pd.DataFrame()

# 将数据集划分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# 其他代码保持不变

# 在循环中，修改模型名称与步骤名称的一致性
for name, model in models.items():
    if name in param_grids:
        # 创建管道
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            (name.lower(), model)
        ])
        # 设置网格搜索
        grid = GridSearchCV(pipeline, param_grid=param_grids[name], cv=5, n_jobs=-1, scoring='neg_root_mean_squared_error', refit=True)
        grid.fit(X_train, y_train)
        best_model = grid.best_estimator_
        best_params = grid.best_params_
    else:
        # 创建管道
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            (name.lower(), model)
        ])
        pipeline.fit(X_train, y_train)
        best_model = pipeline
        best_params = 'N/A'

    # 在验证集上进行预测
    y_pred = best_model.predict(X_test)

    # 计算性能指标
    rmse = root_mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    medae = median_absolute_error(y_test, y_pred)
    # r2 = r2_score(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)

    # 将结果添加到 DataFrame
    new_row = pd.DataFrame({
        'Model': [name],
        'Best Params': [best_params],
        'RMSE': [rmse],
        'MAE': [mae],
        'MeAE': [medae],
        # 'R-squared': [r2],
        'MAPE': [mape]
    })
    results = pd.concat([results, new_row], ignore_index=True)

    # 如果模型具有系数属性，保存系数
    if hasattr(best_model.named_steps[name.lower()], 'coef_'):
        coef = best_model.named_steps[name.lower()].coef_
        coef_df[name] = coef

# 设置系数 DataFrame 的索引
coef_df.index = X.columns
coef_df.index.name = 'Variables'

# 输出结果
print(results)
print(coef_df)


In [ ]:
import os
coef_df_tr = coef_df.transpose()
os.makedirs('./output', exist_ok=True)
coef_df_tr.to_csv(f'./output/{sentiment_label}_coef_df_tr.csv')
results.to_csv(f'./output/{sentiment_label}_results.csv')
print(f'Coef DataFrame and Results saved to output/{sentiment_label}_*.csv')

# 利用最佳模型进行分析

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

# SHAP 与 PDP 相关包
import shap
from pdpbox import pdp

# 假设 X, y 已经定义
# 如果 X 是 numpy 数组，可以考虑转换为 DataFrame，例如：
# X = pd.DataFrame(X, columns=["feat1", "feat2", ...])

# 分割数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 定义并训练决策树模型
best_dt = DecisionTreeRegressor(max_depth=4, min_samples_split=20)
best_dt.fit(X_train, y_train)
y_pred = best_dt.predict(X_test)


In [ ]:

# ========= SHAP 分析 =========
# 使用 TreeExplainer 来解释决策树模型
explainer = shap.TreeExplainer(best_dt)
# 计算 SHAP 值，这里对测试集数据进行解释
shap_values = explainer.shap_values(X_test)
df = pd.concat([pd.DataFrame(X_test), pd.DataFrame(y_test)], axis=1)
shap.initjs()

shap_values = explainer.shap_values(df)
shap.summary_plot(shap_values, df, plot_type='bar', show=False)

# 绘制 SHAP 特征重要性图

shap_sum = np.abs(shap_values).mean(axis=0)
importance_df = pd.DataFrame([data.columns.tolist(), shap_sum.tolist()]).T
importance_df.columns = ['column_name', 'shap_importance']
importance_df = importance_df.sort_values('shap_importance', ascending=False)
importance_df.to_csv('dt_SHAP_feature_importance.csv', index=False)


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# 读取CSV文件
data = pd.read_csv('c:\\Software\\Local Things (Coding)\\comp\\2025大学生建模比赛\\代码\\比赛项目代码\\dt_SHAP_feature_importance.csv')

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei']  # 使用黑体字体
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题

# 设置图形大小
plt.figure(figsize=(10, 6))

# 绘制条形图
plt.bar(data['column_name'], data['shap_importance'], color='skyblue')

# 添加标题和标签
plt.title('各个富豪对于股市波动率的SHAP特征重要性')
plt.xlabel('富豪推文情感标签')
plt.ylabel('SHAP特征重要性')

# 旋转x轴标签以便更好地显示
plt.xticks(rotation=45, ha='right')
plt.savefig('dt_SHAP_feature_importance.png', dpi=300)
# 显示图形
plt.tight_layout()
plt.show()
